In [1]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
builder = (SparkSession.builder
           .appName("config-streaming")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")   
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aeed2286-e47d-4386-8ff6-6420007d1478;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (664ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.4.0!delta-storage.jar (51ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (85ms)
:: resolution report :: resolve 1938ms :: artifacts dl 809ms
	:

In [7]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [8]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [9]:
# Generate running word count
wordCounts = words.groupBy("word").count()

In [10]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Apache|    1|
| Spark|    1|
|Hadoop|    1|
+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Apache|    2|
| Spark|    2|
|Hadoop|    1|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Apache|    3|
| Kafka|    1|
| Spark|    2|
|Hadoop|    1|
+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|    Apache|    3|
|     Kafka|    1|
|Structured|    1|
| Streaming|    1|
|     Spark|    2|
|    Hadoop|    1|
+----------+-----+



In [ ]:
query.awaitTermination()

In [11]:
spark.stop()